In [1]:
import csv, os, zipfile, functools, datetime, re
import pandas as pd
import numpy as np

In [2]:
#CONSTANTS

KEEP_COLUMNS = {
    "play_id": "string",
    "game_id": "string",
    "home_team": "string",
    "away_team": "string",
    "posteam": "string",
    "posteam_type": "string",
    "side_of_field": "string",
    "yardline_100": "Int64",
    "game_date": datetime,
    "half_seconds_remaining": "Int64",
    "game_seconds_remaining": "Int64",
    "game_half": "string",
    "drive": "Int64",
    "down": "string", 
    "goal_to_go": "Int64", 
    "ydsnet": "Int64",
    "desc": "string",
    "ydstogo": "Int64", 
    "play_type": "string",
    "yards_gained": "Int64",
    "shotgun": 	"Int64", 
    "no_huddle": 	"Int64",
    "qb_dropback": 	"Int64",
    "qb_scramble": "Int64",
    "pass_length": "Int64", 
    "pass_location": "string",
    "run_location": "string",
    "run_gap": "string", 
    "posteam_timeouts_remaining": "Int64", 
    "defteam_timeouts_remaining": "Int64", 
    "posteam_score": "Int64", 
    "defteam_score": "Int64",
    "score_differential": "Int64",

}

In [3]:
#Data Source: https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016

plays = pd.read_csv("files/input/NFL Play by Play 2009-2018 (v5).csv")
plays.sample(1000).to_csv("files/output/full_preview.csv", index=False)
#print("\n".join(f"{i}. {c}" for i,c in enumerate(plays.columns)))
plays[KEEP_COLUMNS.keys()].sample(1000).to_csv("files/output/filtered_preview.csv", index=False)
plays[plays["game_id"] == 2009091000].to_csv("files/output/one_game_preview.csv", index=False)
print(len(plays))
plays["play_id"] = range(1, len(plays) + 1)



/var/folders/0p/p8g1hvy94_q9bd228nd5dfyr0000gn/T/ipykernel_78521/2856056326.py:3: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  plays = pd.read_csv("files/input/NFL Play by Play 2009-2018 (v5).csv")


449371


In [4]:
#Cleanup

#REMOVE UINTERESTIN COLUMNS
plays = plays[KEEP_COLUMNS.keys()]
for col,dtype in KEEP_COLUMNS.items():
    if dtype is datetime:
        plays[col] = pd.to_datetime(plays[col])
    else:
        plays[col] = plays[col].astype(dtype, errors="ignore")
        
plays["play_type"] = plays["play_type"].fillna("no_play")
plays["qb_dropback"] = plays["qb_dropback"].fillna(0)
#need simple play type run vs pass
#need complex play type run left, run middle, run right, pass short left, pass short right, pass short middle, pass long left, pass long right, pass long middle
#need Cumulative percentage run per game
#need cumulative percentage run per season 
#need cumulative yards per pass play per game
#need cumulative yards per run play per game
#need cumulative yards per pass play per season
#need cumulative yards per run play per season
#CHECK IF PASSING PLAYS SHOW UP AS RUNNING PLAYS ON SCRAMBLE, USE QB DROP BACK


#REMOVE UNINTERESTING ROWS

In [5]:
#plays["play_type"].value_counts()

filtered = plays[
    (plays["pass_location"].isna() & ~plays["pass_length"].isna()) |
    (~plays["pass_location"].isna() & plays["pass_length"].isna())
]
print(len(filtered))
filtered.head(5)


0


,play_id,game_id,home_team,away_team,posteam,posteam_type,side_of_field,yardline_100,game_date,half_seconds_remaining,...,qb_scramble,pass_length,pass_location,run_location,run_gap,posteam_timeouts_remaining,defteam_timeouts_remaining,posteam_score,defteam_score,score_differential


In [6]:
#filtered[["pass_location", "pass_length"]].isna().value_counts()
#filtered[["pass_location", "pass_length"]].head(10).to_dict("records")

#plays[plays["qb_dropback"].isna()].head(5)[["play_id", "desc"]]


In [7]:


#GET THE SEASON SO IT CAN BE USED FOR AGGREGATIONS
def determine_season(game_date):
    year = game_date.year
    if game_date.month < 8:
        return str(year - 1)
    return year
if "season" not in plays.columns:
    game_seasons = dict(( play[0], determine_season(play[1])) for play in plays[["game_id", "game_date"]].drop_duplicates().values.tolist())
    plays["season"] = plays["game_id"].apply(lambda x: game_seasons[x])


#GET THE TYPES OF PLAYS. THES WILL BE THE RESPONSE/DEPENDENT VARIABLES
plays[["pass_length", "pass_location", "run_location", "play_type"]] = plays[["pass_length", "pass_location", "run_location", "play_type"]].fillna("")
def determine_play_types(row):
    play_type_simple,play_type_adv = "",""
    if row["play_type"] == "no_play":
        return [play_type_simple, play_type_adv]   

    if row["qb_dropback"]:
        play_type_simple = "pass"
        if row["qb_scramble"]:
            play_type_adv = "scramble"
    elif row["play_type"] == "run":
        play_type_simple = "run"

    if row["run_location"]:
        play_type_adv = "run_" + row["run_location"]
    if row["pass_length"] and row["pass_location"]:
        play_type_adv = "pass_" + row["pass_length"] + "_" + row["pass_location"]
    if row["qb_scramble"]:
        play_type_adv = "pass_scramble"
    elif row["qb_dropback"] and not row["pass_length"]:
        play_type_adv = "pass_failure"
    return [play_type_simple, play_type_adv]
full_play_types = plays.apply(lambda row: determine_play_types(row), axis=1)
 
plays[["play_type_simple", "play_type_adv"]] = pd.DataFrame(full_play_types.to_list(), index=plays.index)
#Can Drop Irrelevant Plays Now
plays = plays[plays["play_type_simple"].isin(["run", "pass"])]




In [8]:
"""
Decided not to use this.
game_aggregated = plays.groupby(["game_id", "posteam"], group_keys=False)
#DETERMINE IF THE PREVIOUS PLAY WAS A TIMEOUT
def check_timeout_prior(group):
    prev_desc = group["desc"].str.lower().shift(1)
    group["timeout_prior"] = prev_desc.str.contains("timeout", na=False)
    return group
game_aggregated = game_aggregated.apply(check_timeout_prior)
timeout_flags = game_aggregated[["play_id", "timeout_prior"]]
plays = plays.merge(timeout_flags, on="play_id")
plays.to_csv("files/output/checkpoint.csv", index=False)
"""

'\nDecided not to use this.\ngame_aggregated = plays.groupby(["game_id", "posteam"], group_keys=False)\n#DETERMINE IF THE PREVIOUS PLAY WAS A TIMEOUT\ndef check_timeout_prior(group):\n    prev_desc = group["desc"].str.lower().shift(1)\n    group["timeout_prior"] = prev_desc.str.contains("timeout", na=False)\n    return group\ngame_aggregated = game_aggregated.apply(check_timeout_prior)\ntimeout_flags = game_aggregated[["play_id", "timeout_prior"]]\nplays = plays.merge(timeout_flags, on="play_id")\nplays.to_csv("files/output/checkpoint.csv", index=False)\n'

In [12]:

plays["season_cumul_run"] = (
    plays["play_type_simple"].eq("run")
    .groupby([plays["season"], plays["posteam"]])
    .cumsum()
)
plays["season_cumul_run_yds"] = (
    plays["yards_gained"]
    .where(plays["play_type_simple"] == "run", 0)
    .groupby([plays["season"], plays["posteam"]])
    .cumsum()
)
plays["season_cumul_pass"] = (
    plays["play_type_simple"].eq("pass")
    .groupby([plays["season"], plays["posteam"]])
    .cumsum()
)
plays["season_cumul_pass_yds"] = (
    plays["yards_gained"]
    .where(plays["play_type_simple"] == "pass", 0)
    .groupby([plays["season"], plays["posteam"]])
    .cumsum()
)
plays["season_cumul_run_avg"] = plays["season_cumul_run_yds"] / plays["season_cumul_run"]
plays["season_cumul_pass_avg"] = plays["season_cumul_pass_yds"] / plays["season_cumul_pass"]





#season_aggregated = plays.groupby(["season", "posteam"])



#plays = pd.merge()


319369

436129

In [ ]:
#TRAINING SET 1 FIRST 80% OF EACH GAME
#TRAININT SET 2 FIRST 80% OF EACH SEASON